--- 

<h1 style="color: #D97C00">KNAPSACK 0/1</h1>

---

- Mô tả: Bạn có một balo với một sức chứa nhất định và một tập hợp các vật phẩm. Mỗi vật phẩm có một trọng lượng và một giá trị. 
- Mục tiêu là chọn một tập hợp con các vật phẩm sao cho tổng trọng lượng không vượt quá sức chứa của balo, đồng thời tổng giá trị là lớn nhất.
- Đặc điểm: Mỗi vật phẩm chỉ có thể chọn hoặc không chọn (0 hoặc 1).


<h3 style="color:#F9E15B"> Thư viện Random </h3>    

- Tạo các cá thể ban đầu một cách ngẫu nhiên.

In [1]:
import random

<h3 style="color:#F9E15B">I. Các thông số tối ưu của GA</h3>

- Kích thước quần thể. (Population Size)
- Số thế hệ. (Generations)
- Tỷ lệ lai ghép. (Crossover Rate)
- Tỷ lệ đột biến. (Mutation Rate)

In [ ]:
POPULATION_SIZE = 200  # Kích thước quần thể
GENERATIONS = 200  # Số thế hệ
CROSSOVER_RATE = 0.8  # Tỷ lệ lai ghép
MUTATION_RATE = 0.04  # Tỷ lệ đột biến

<h3 style="color:#F9E15B">II. Các thông số của Knapsack problem</h3>

- Danh sách chứa thông tin về các vật phẩm trong bài toán cái ba lô, bao gồm trọng lượng và giá trị của mỗi vật phẩm.
- Dung lượng tối đa của cái ba lô trong bài toán.
- Danh sách lưu trữ giá trị fitness của các cá thể tốt nhất qua từng thế hệ.

In [11]:
items = [] # Vật phẩm chứa weights và values
max_capacity = 0 # maximum capacity
fitness_history = [] # Chứa lịch sử fitness qua các thế hệ

<h3 style="color:#F9E15B">III. Khởi tạo quần thể</h3>  

- Quần thể được tạo ngẫu nhiên, mỗi cá thể biểu diễn một giải pháp tiềm năng thông qua chuỗi gene nhị phân (0 và 1).

In [7]:
def initialize_population(num_items):
    return [[random.randint(0, 1) for _ in range(num_items)] for _ in range(POPULATION_SIZE)]

<h3 style="color:#F9E15B">IV. Tính toán giá trị fitness của cá thể dựa trên trọng lượng và giá trị vật phẩm</h3>

- Fitness của mỗi cá thể được tính dựa trên số vật phẩm được chọn và tính tổng value của số vật phẩm được chọn.
- Nếu cá thể có fitness > dung lượng của balo thì fitness sẽ được gán 0.

In [6]:
def fitness(individual):
    total_weight = sum(individual[i] * items[i][0] for i in range(len(items)))
    total_value = sum(individual[i] * items[i][1] for i in range(len(items)))
    return total_value if total_weight <= max_capacity else 0

<h3 style="color:#F9E15B">V. Elitism Selection.</h3>

- Sử dụng chọn lọc ưu tú.
- Chọn cá thể có fitness cao nhất để giữ lại.
- Các cá thể khác được chọn theo thứ tự giảm dần fitness.

In [8]:
def select_population(population):
    sorted_population = sorted(population, key=lambda x: fitness(x), reverse=True)
    return sorted_population[:POPULATION_SIZE]

<h3 style="color:#F9E15B">VI. Uniform Crossover.</h3>

- Theo cơ chế tung đồng xu (0,1).
- Mỗi lần tung đồng xu, duyệt qua từng phần tử cá thể con.
- Thay thế gán mỗi bit bằng bit của cha hoặc mẹ tuỳ vào thiết lập.

In [ ]:
def crossover(parent1, parent2):
    child1, child2 = parent1[:], parent2[:]
    for i in range(len(parent1)):
        coin_flip = random.choice([0, 1])
        if coin_flip == 0:
            child1[i] = parent1[i]
            child2[i] = parent2[i]
        else:
            child1[i] = parent2[i]
            child2[i] = parent1[i]
    return child1, child2  

<h3 style="color:#F9E15B">VII. Bit-flip mutation.</h3>

- Sử dụng đột biến thông qua cách sử dụng tới thông số tỉ lệ đột biến.
- Duyệt qua từng bit trong cá thể, nếu số random nhỏ hơn thông số tỉ lệ đột biến thì bit bị lật (0 thành 1 hoặc 1 thành 0).

In [ ]:
def mutate(individual):
    for i in range(len(individual)):
        if random.random() < MUTATION_RATE:
            individual[i] = 1 - individual[i]
    return individual

<h3 style="color:#F9E15B">VIII. Triển khai thuật toán</h3>  

Lặp qua các thế hệ: Trong mỗi thế hệ, thực hiện:

- Chọn lọc quần thể dựa trên fitness.
- Lai ghép các cặp cá thể để tạo thế hệ con với xác suất lai ghép.
- Thực hiện đột biến trên các cá thể con để tăng tính đa dạng.
- Cập nhật quần thể và cá thể tốt nhất nếu có cá thể mới vượt trội.  

Lưu giá trị fitness tốt nhất qua từng thế hệ để đánh giá quá trình tiến hóa.  

Trả về kết quả cá thể tốt nhất và giá trị fitness tương ứng 

In [ ]:
def genetic_algorithm():
    global fitness_history
    
    population = initialize_population(len(items))
    best_individual = max(population, key=lambda ind: fitness(ind))

    for gen in range(GENERATIONS):
        selected_population = select_population(population)
        
        num_to_crossover = int(POPULATION_SIZE * CROSSOVER_RATE)
        selected_for_crossover = selected_population[:num_to_crossover]
        
        offspring = []
        for i in range(0, num_to_crossover, 2):
            if i + 1 < len(selected_for_crossover):
                child1, child2 = crossover(selected_for_crossover[i], selected_for_crossover[i + 1])
                offspring.extend([child1, child2])
        
        offspring = [mutate(ind) for ind in offspring]
        
        # Cắt tỉa quần thể để đảm bảo kích thước không vượt quá POPULATION_SIZE
        population.extend(offspring)
        population = sorted(population, key=lambda ind: fitness(ind), reverse=True)
        population = population[:POPULATION_SIZE]
        
        current_best = max(population, key=lambda ind: fitness(ind))
        fitness_history.append(fitness(current_best))
        
        if fitness(current_best) > fitness(best_individual):
            best_individual = current_best

    return best_individual, fitness(best_individual)

--- 

<h2 style="color:#D97C00"> Tổng thể </h2>

---

- Bài toán trên được thiết kế để tối ưu hóa trong không gian tìm kiếm rộng lớn và phức tạp, nơi các phương pháp giải quyết truyền thống gặp khó khăn.  
- Với các khả năng tìm kiếm toàn cục hiệu quả, tránh bị kẹt trong các nghiệm cục bộ nhờ vào sự kết hợp giữa các cơ chế Selection, Crossover, và Mutation.  